# Udacity_project_01

## Workspace setup

In [1]:
from azureml.core import Workspace, Experiment

# Get the existing workspace
ws = Workspace.from_config()#get(name="quick-starts-ws-139918", resource_group="aml-quickstarts-139918" )

# Verify workspace attributes
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-139949
Azure region: southcentralus
Subscription id: 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group: aml-quickstarts-139949


## Experiment setup

In [2]:
# Create the experiment
exp = Experiment(workspace=ws, name="Udacity_project_01_Exp01")
run = exp.start_logging()

## ComputeTarget setup

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Name the compute cluster
cluster_name = "MyComputeInstanceJDY"

# Verify that name do not exist already
try:
    comp_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Create new cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    comp_target = ComputeTarget.create(ws, cluster_name, compute_config)

comp_target.wait_for_completion(show_output=True)
print('Cluster Details: ', comp_target.get_status().serialize())

Found existing cluster, use it.

Running
Cluster Details:  {'errors': [], 'creationTime': '2021-03-06T16:24:22.386909+00:00', 'createdBy': {'userObjectId': '0d6538d0-f2fa-4a58-b147-37b298c51455', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': None}, 'modifiedTime': '2021-03-06T16:27:09.842228+00:00', 'state': 'Running', 'vmSize': 'STANDARD_D2_V2'}


## Increase scalability

In [5]:
%%writefile conda_dependencies.yml
# Setup environment config with libraries versions
dependencies:
    - python=3.6.2
    - scikit-learn
    - numpy
    - pip:
        - azureml-defaults

Writing conda_dependencies.yml


In [6]:
from azureml.core import Environment
# Create environment
sklearn_env= Environment.from_conda_specification(name='sklearn_env', file_path='conda_dependencies.yml')

## Run HyperDrive Config

In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"C": uniform(0.01, 1), "max_iter": choice(range(10, 200, 25))})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor = 0.2, delay_evaluation=0)

# Create a ScriptRunConfig for use with train.py
est = ScriptRunConfig(source_directory = "./", script="train.py", compute_target=comp_target, environment=sklearn_env)

# Create a HyperDriveConfig using the run_config, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4
                                    )

In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyper_run = exp.submit(hyperdrive_config)
RunDetails(hyper_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Record best model

In [16]:
import joblib
# Get your best run and save the model from that run.
hd_best_run = hyper_run.get_best_run_by_primary_metric()
hd_best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Udacity_project_01_Exp01,HD_8d1bcc4d-a7f2-450a-ac2a-a63406acc531_5,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Load data set in DatasetFactory

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path=path)

In [12]:
from train import clean_data
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
# Concat X & y-> no train test split is done because the model will use cross validation
data_cleaned = pd.concat([x,y], axis=1)

## Run AutoML Config

In [13]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_cleaned,
    label_column_name='y',
    n_cross_validations=5)

In [14]:
# Submit your automl run
automlrun = exp.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_385c2f14-ea01-4b3f-8009-94833aa03f9f

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

In [15]:
# Retrieve and save your best automl model.
atml_best_run = automlrun.get_best_child()
atml_best_run.get_details()

{'runId': 'AutoML_385c2f14-ea01-4b3f-8009-94833aa03f9f_34',
 'status': 'Completed',
 'startTimeUtc': '2021-03-06T17:12:59.695127Z',
 'endTimeUtc': '2021-03-06T17:13:39.297727Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'Udacity_project_01_Exp01\',\'compute_target\':\'local\',\'subscription_id\':\'2c48c51c-bd47-40d4-abbe-fb8eabd19c8c\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_385c2f14-ea01-4b3f-8009-94833aa03f9f_34","experiment_name":null,"workspace_name":"quick-starts-ws-139949","subscription_id":"2c48c51c-bd47-40d4-abbe-fb8eabd19c8c","resource_group_name"

In [18]:
best_run, fitted_model = automlrun.get_output()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Udacity_project_01_Exp01,AutoML_385c2f14-ea01-4b3f-8009-94833aa03f9f_34,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [19]:
fitted_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                intercept_scaling=1,
                                                                                                l1_ratio=None,
                                                                                                max_iter=100,
                                                      

In [20]:
# Close properly
comp_target.delete()